In [1]:
%run ../__init__.ipynb

In [2]:
df_pockets = read_pockets()
print(df_pockets['pocket_score_bin'].value_counts())
phead(df_pockets)

0 to 200       109481
200 to 400     109480
400 to 600     109480
600 to 800     109480
800 to 1000    109480
Name: pocket_score_bin, dtype: int64
547,401 records


,0,1,2
uniprot_id,A0A024R1R8,A0A024R1R8,A0A024RBG1
struct_id,A0A024R1R8-F1,A0A024R1R8-F1,A0A024RBG1-F1
pocket_id,1,2,1
pocket_energy,50.329,37.488,244.37
pocket_n_points,159.0,108.0,561.0
pocket_rad_gyration,4.042788,3.175737,7.310256
pocket_energy_per_vol,0.316535,0.347111,0.435597
pocket_buriedness,0.772959,0.808219,0.856184
pocket_score,23.497914,22.21455,56.255444
pocket_cl_file,results/23.04_bindfunc/af2.obabel_hxr.autosite...,results/23.04_bindfunc/af2.obabel_hxr.autosite...,results/23.04_bindfunc/af2.obabel_hxr.autosite...


In [3]:
df_af2 = read_structures()

In [4]:
df_clinvar = read_clinvar() #read_Cheng2023_s5()#.head(5)
#df_clinvar[['uniprot_id', 'aa_pos', 'aa_ref', 'aa_alt']] = df_clinvar.apply(lambda r: parse_varstr(r['protein_variant']), axis=1, result_type='expand')

df_pathogenic = df_clinvar.query('clinvar_label == "Pathogenic"').groupby('Uniprot_id').agg(aa_pos = ('Amino_acid_position', lambda x: set((int(x_i) for x_i in x))))#.reset_index()
df_benign = df_clinvar.query('clinvar_label == "Benign"').groupby('Uniprot_id').agg(aa_pos = ('Amino_acid_position', lambda x: set((int(x_i) for x_i in x))))#.reset_index()

/cluster/project/beltrao/jjaenes/af2genomics/src/af2genomics/__init__.py:954: DtypeWarning: Columns (0,24,25,26,27,28,29,30,31,32,33,34,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ = pd.read_csv(fp_, sep='\t', *args, **kwargs)


read_clinvar: 1,857,854 rows from /cluster/work/beltrao/jjaenes/23.06.02_clinvar/24.04.22_protvar_out/clinvar_mapped.tsv
read_clinvar: 1,761,408 after removing conflicting/other variants


In [5]:
df_pharmgkb = pd.read_csv('/cluster/project/beltrao/jjaenes/22.12_pocketomes/resources/pharmgkb/clinicalAnnotations_mapped.tsv', sep='\t')#.query(q_)
printlen(df_pharmgkb, 'pharmgkb variants')
df_pharmgkb_resid = df_pharmgkb.groupby('Uniprot_canonical_isoform_(non_canonical)').agg(aa_pos = ('Amino_acid_position', lambda x: set((int(x_i) for x_i in x))))#.reset_index()
printlen(df_pharmgkb_resid, 'genes with pharmgkb variants')

1,834 pharmgkb variants
425 genes with pharmgkb variants


In [6]:
df_ddd = read_Cheng2023_s6()
df_ddd[['uniprot_id', 'aa_pos', 'aa_ref', 'aa_alt']] = df_ddd.apply(lambda r: parse_varstr(r['protein_variant']), axis=1, result_type='expand')
printlen(df_ddd, 'DDD variants')
df_ddd_path_resid = df_ddd.query('label == 1').groupby('uniprot_id').agg(aa_pos = ('aa_pos', lambda x: set((int(x_i) for x_i in x))))#.reset_index()
df_ddd_bngn_resid = df_ddd.query('label == 0').groupby('uniprot_id').agg(aa_pos = ('aa_pos', lambda x: set((int(x_i) for x_i in x))))#.reset_index()
df_ddd_path_resid
#df_ddd_bngn_resid

2,601 DDD variants


,aa_pos
uniprot_id,
O00329,"{416, 1021}"
O00459,"{373, 557}"
O00743,"{264, 270}"
O14497,"{1989, 2087, 2158, 1461, 1276}"
O14522,"{364, 668, 548, 695}"
...,...
Q9Y2L1,"{488, 479, 780, 689, 382, 767}"
Q9Y448,{22}
Q9Y4H2,{1057}


In [7]:
df_path_resid = pd.concat([df_pathogenic['aa_pos'].rename('clinvar_path'), df_ddd_path_resid['aa_pos'].rename('ddd_path'), df_pharmgkb_resid['aa_pos'].rename('pharmgkb')], axis=1)
df_path_resid['clinvar_path'] = fillna_set(df_path_resid['clinvar_path'])
df_path_resid['ddd_path'] = fillna_set(df_path_resid['ddd_path'])
df_path_resid['pharmgkb'] = fillna_set(df_path_resid['pharmgkb'])
df_path_resid['pathogenic_resid'] = df_path_resid.apply(lambda r: r['clinvar_path'] | r['ddd_path'] | r['pharmgkb'], axis=1)
df_path_resid

,clinvar_path,ddd_path,pharmgkb,pathogenic_resid
A0A087WTJ0,"{1, 214}",{},{},"{1, 214}"
A0A087WV20,{3850},{},{},{3850}
A0A087WV90,"{753, 258, 861, 471}",{},{},"{753, 258, 861, 471}"
A0A087WWA2,{60},{},{},{60}
A0A087WX10,{385},{},{},{385}
...,...,...,...,...
Q9Y694,{},{},{425},{425}
Q9Y6D9,{},{},{558},{558}
Q9Y6L6,{},{},"{130, 155, 174, 199}","{130, 155, 174, 199}"
R4GN13,{},{},{112},{112}


In [8]:
df_bngn_resid = pd.concat([df_benign['aa_pos'].rename('clinvar_bngn')], axis=1).rename({'clinvar_bngn': 'benign_resid'}, axis=1)
df_bngn_resid

,benign_resid
Uniprot_id,
A0A024R4E5,"{262, 1177, 418, 1197, 190, 831, 192, 1092, 46..."
A0A075B6E9,"{202, 86}"
A0A075B718,"{148, 150, 151}"
A0A075B725,{71}
A0A075B759,{69}
...,...
X6REP5,"{421, 414}"
X6REW1,"{720, 2, 83, 307}"
X6RGM1,{45}


In [9]:
df_ = df_pockets.merge(df_af2, left_on='uniprot_id', right_on='uniprot_id').merge(df_path_resid, left_on='uniprot_id', right_index=True).merge(df_bngn_resid, left_on='uniprot_id', right_index=True, how='left')
df_['benign_resid'] = fillna_set(df_['benign_resid'])
df_

,uniprot_id,struct_id,pocket_id,pocket_energy,pocket_n_points,pocket_rad_gyration,pocket_energy_per_vol,pocket_buriedness,pocket_score,pocket_cl_file,...,entrez_id,isin_CRISPRInferredCommonEssentials,isin_AchillesCommonEssentialControls,isin_AchillesNonessentialControls,struct_resid,clinvar_path,ddd_path,pharmgkb,pathogenic_resid,benign_resid
3888,A0A2R8Y5M8,A0A2R8Y5M8-F1,1,327.808,762.0,9.584846,0.430194,0.843858,56.612067,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,NaN,False,False,False,"{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",{48},{},{},{48},"{35, 100, 165, 6, 7, 169, 203, 175, 146, 254, ..."
3889,A0A2R8Y5M8,A0A2R8Y5M8-F1,2,117.391,306.0,5.401413,0.383631,0.795238,35.826830,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,NaN,False,False,False,"{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",{48},{},{},{48},"{35, 100, 165, 6, 7, 169, 203, 175, 146, 254, ..."
3890,A0A2R8Y5M8,A0A2R8Y5M8-F1,3,71.538,192.0,3.868656,0.372594,0.793765,31.269793,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,NaN,False,False,False,"{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",{48},{},{},{48},"{35, 100, 165, 6, 7, 169, 203, 175, 146, 254, ..."
3891,A0A2R8Y5M8,A0A2R8Y5M8-F1,4,71.115,164.0,4.035254,0.433628,0.847666,29.202654,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,NaN,False,False,False,"{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",{48},{},{},{48},"{35, 100, 165, 6, 7, 169, 203, 175, 146, 254, ..."
3892,A0A2R8Y5M8,A0A2R8Y5M8-F1,5,85.721,226.0,4.946703,0.379296,0.798419,29.124221,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,NaN,False,False,False,"{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",{48},{},{},{48},"{35, 100, 165, 6, 7, 169, 203, 175, 146, 254, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547218,Q9Y6Y1,Q9Y6Y1-F1,56,40.767,136.0,3.593565,0.299757,0.732704,20.317532,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,23261.0,False,False,False,"{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{1473, 163, 135, 141, 78, 1077, 886, 955}",{},{},"{1473, 163, 135, 141, 78, 1077, 886, 955}","{1026, 5, 517, 1037, 527, 17, 1041, 534, 538, ..."
547219,Q9Y6Y1,Q9Y6Y1-F1,57,43.909,140.0,3.923867,0.313636,0.743284,19.711642,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,23261.0,False,False,False,"{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{1473, 163, 135, 141, 78, 1077, 886, 955}",{},{},"{1473, 163, 135, 141, 78, 1077, 886, 955}","{1026, 5, 517, 1037, 527, 17, 1041, 534, 538, ..."
547220,Q9Y6Y1,Q9Y6Y1-F1,58,40.964,128.0,3.942287,0.320031,0.777126,19.608516,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,23261.0,False,False,False,"{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{1473, 163, 135, 141, 78, 1077, 886, 955}",{},{},"{1473, 163, 135, 141, 78, 1077, 886, 955}","{1026, 5, 517, 1037, 527, 17, 1041, 534, 538, ..."
547221,Q9Y6Y1,Q9Y6Y1-F1,59,43.446,142.0,4.007809,0.305958,0.742775,19.547685,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,23261.0,False,False,False,"{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{1473, 163, 135, 141, 78, 1077, 886, 955}",{},{},"{1473, 163, 135, 141, 78, 1077, 886, 955}","{1026, 5, 517, 1037, 527, 17, 1041, 534, 538, ..."


In [10]:
#df_['n_pocket_resid_pathogenic'] = df_.apply(lambda r: len(set(r['pocket_resid']) & r['pathogenic_resid']), axis=1)
#df_['n_pocket_resid_not_pathog'] = df_.apply(lambda r: len(set(r['pocket_resid']) - r['pathogenic_resid']), axis=1)
#df_['n_others_resid_pathogenic'] = df_.apply(lambda r: len(set(r['struct_resid']) & r['pathogenic_resid'] - set(r['pocket_resid'])), axis=1)
#df_['n_others_resid_not_pathog'] = df_.apply(lambda r: len(set(r['struct_resid']) - r['pathogenic_resid'] - set(r['pocket_resid'])), axis=1)
#df_pval_ = df_.query('n_pocket_resid_pathogenic > n_others_resid_pathogenic').copy()

def fisher_exact_(resid_var, resid_feat, resid_all):
    """
        resid_var: residues with a variant
        resid_feat: residues with a feature
        resid_all: all residues
    """
    k = len(resid_feat & resid_var) # n_pocket_resid_pathogenic
    l = len(resid_feat - resid_var) # n_pocket_resid_not_pathog
    m = len(resid_all & resid_var - resid_feat) # n_others_resid_pathogenic
    n = len(resid_all - resid_var - resid_feat) # n_others_resid_not_pathog
    (odds, p) = sp.stats.fisher_exact([[k, l], [m, n]])
    return (k, l, m, n, odds, p)

#df_pval_[['statistic', 'pvalue']] = df_pval_.apply(lambda r: fisher_exact_(
#    r['n_pocket_resid_pathogenic'],
#    r['n_pocket_resid_not_pathog'],
#    r['n_others_resid_pathogenic'],
#    r['n_others_resid_not_pathog'],
#), axis=1, result_type='expand')

#df_pval_['reject'], df_pval_['pvals_corrected'], df_pval_['alphacSidak'], df_pval_['alphacBonf'] =\
#    statsmodels.stats.multitest.multipletests(df_pval_['pvalue'], method='fdr_bh')

#printlen(df_pval_.query('pvals_corrected < .05'), 'pockets with enrichment after FDR correction')
#df_pval_

cols_ = ['n_pocket_resid_pathogenic', 'n_pocket_resid_not_pathogenic', 'n_others_resid_pathogenic', 'n_others_resid_not_pathogenic', 'pathogenic_odds', 'pathogenic_pvalue']
df_[cols_] = df_.apply(lambda r: fisher_exact_(r.pathogenic_resid, r.pocket_resid, r.struct_resid), axis=1, result_type='expand')

cols_ = ['n_pocket_resid_benign', 'n_pocket_resid_not_benign', 'n_others_resid_benign', 'n_others_resid_not_benign', 'benign_odds', 'benign_pvalue']
df_[cols_] = df_.apply(lambda r: fisher_exact_(r.benign_resid, r.pocket_resid, r.struct_resid), axis=1, result_type='expand')
df_

,uniprot_id,struct_id,pocket_id,pocket_energy,pocket_n_points,pocket_rad_gyration,pocket_energy_per_vol,pocket_buriedness,pocket_score,pocket_cl_file,...,n_others_resid_pathogenic,n_others_resid_not_pathogenic,pathogenic_odds,pathogenic_pvalue,n_pocket_resid_benign,n_pocket_resid_not_benign,n_others_resid_benign,n_others_resid_not_benign,benign_odds,benign_pvalue
3888,A0A2R8Y5M8,A0A2R8Y5M8-F1,1,327.808,762.0,9.584846,0.430194,0.843858,56.612067,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,1.0,204.0,0.0,1.0,1.0,52.0,14.0,191.0,0.262363,0.319346
3889,A0A2R8Y5M8,A0A2R8Y5M8-F1,2,117.391,306.0,5.401413,0.383631,0.795238,35.826830,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,1.0,242.0,0.0,1.0,0.0,15.0,15.0,228.0,0.000000,1.000000
3890,A0A2R8Y5M8,A0A2R8Y5M8-F1,3,71.538,192.0,3.868656,0.372594,0.793765,31.269793,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,1.0,243.0,0.0,1.0,4.0,10.0,11.0,233.0,8.472727,0.005310
3891,A0A2R8Y5M8,A0A2R8Y5M8-F1,4,71.115,164.0,4.035254,0.433628,0.847666,29.202654,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,1.0,239.0,0.0,1.0,0.0,18.0,15.0,225.0,0.000000,0.608859
3892,A0A2R8Y5M8,A0A2R8Y5M8-F1,5,85.721,226.0,4.946703,0.379296,0.798419,29.124221,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,1.0,238.0,0.0,1.0,3.0,16.0,12.0,227.0,3.546875,0.087519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547218,Q9Y6Y1,Q9Y6Y1-F1,56,40.767,136.0,3.593565,0.299757,0.732704,20.317532,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,8.0,1660.0,0.0,1.0,1.0,4.0,170.0,1498.0,2.202941,0.417127
547219,Q9Y6Y1,Q9Y6Y1-F1,57,43.909,140.0,3.923867,0.313636,0.743284,19.711642,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,8.0,1656.0,0.0,1.0,0.0,9.0,171.0,1493.0,0.000000,0.610609
547220,Q9Y6Y1,Q9Y6Y1-F1,58,40.964,128.0,3.942287,0.320031,0.777126,19.608516,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,8.0,1658.0,0.0,1.0,2.0,5.0,169.0,1497.0,3.543195,0.154915
547221,Q9Y6Y1,Q9Y6Y1-F1,59,43.446,142.0,4.007809,0.305958,0.742775,19.547685,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,8.0,1657.0,0.0,1.0,0.0,8.0,171.0,1494.0,0.000000,1.000000


In [11]:
# FDR-corrected p-values with indepenent filtering for more variants within the pocket region vs rest-of-the-protein
m_ = (df_['pocket_score_combined_scaled'] > 800) & (df_['n_pocket_resid_pathogenic'] > df_['n_others_resid_pathogenic'])
df_.loc[m_, 'pathogenic_padj'] = statsmodels.stats.multitest.multipletests(df_.loc[m_, 'pathogenic_pvalue'], method='fdr_bh')[1]
m_ = (df_['pocket_score_combined_scaled'] > 800) & (df_['n_pocket_resid_benign'] > df_['n_others_resid_benign'])
df_.loc[m_, 'benign_padj'] = statsmodels.stats.multitest.multipletests(df_.loc[m_, 'benign_pvalue'], method='fdr_bh')[1]
printlen(df_.query('pathogenic_padj < .05'), 'pockets enriched for pathogenic variants')
printlen(df_.query('benign_padj < .05'), 'pockets enriched for benign variants')

113 pockets enriched for pathogenic variants
1 pockets enriched for benign variants


In [12]:
df_.query('struct_id == "P12235-F1"')

,uniprot_id,struct_id,pocket_id,pocket_energy,pocket_n_points,pocket_rad_gyration,pocket_energy_per_vol,pocket_buriedness,pocket_score,pocket_cl_file,...,pathogenic_odds,pathogenic_pvalue,n_pocket_resid_benign,n_pocket_resid_not_benign,n_others_resid_benign,n_others_resid_not_benign,benign_odds,benign_pvalue,pathogenic_padj,benign_padj
103968,P12235,P12235-F1,1,177.963,378.0,5.339499,0.470802,0.883355,55.241063,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,22.200000,0.007041,4.0,34.0,47.0,213.0,0.533166,0.355965,0.042244,NaN
103969,P12235,P12235-F1,2,91.658,216.0,4.530073,0.424343,0.845188,34.060850,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,0.000000,1.000000,2.0,16.0,49.0,231.0,0.589286,0.747582,NaN,NaN
103970,P12235,P12235-F1,3,111.422,321.0,5.433295,0.347109,0.756019,33.768165,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,0.000000,1.000000,4.0,19.0,47.0,228.0,1.021277,1.000000,NaN,NaN
103971,P12235,P12235-F1,4,70.452,172.0,3.684785,0.409605,0.848958,33.642557,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,0.000000,1.000000,3.0,14.0,48.0,233.0,1.040179,1.000000,NaN,NaN
103972,P12235,P12235-F1,5,124.581,368.0,6.471922,0.338535,0.756906,32.576058,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,0.000000,1.000000,1.0,23.0,50.0,224.0,0.194783,0.092594,NaN,NaN
103973,P12235,P12235-F1,6,90.060,236.0,4.587874,0.381610,0.794549,32.474480,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,0.000000,1.000000,2.0,14.0,49.0,233.0,0.679300,1.000000,NaN,NaN
103974,P12235,P12235-F1,7,70.144,199.0,3.848387,0.352482,0.754430,29.431520,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,0.000000,1.000000,2.0,14.0,49.0,233.0,0.679300,1.000000,NaN,NaN
103975,P12235,P12235-F1,8,70.644,197.0,4.416804,0.358599,0.790254,27.854271,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,0.000000,1.000000,3.0,13.0,48.0,234.0,1.125000,0.742359,NaN,NaN
103976,P12235,P12235-F1,9,68.235,227.0,4.651348,0.300595,0.729258,25.954280,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,6.666667,0.187519,3.0,12.0,48.0,235.0,1.223958,0.727357,NaN,NaN
103977,P12235,P12235-F1,10,52.562,78.0,2.904490,0.673872,0.971660,25.354398,results/23.04_bindfunc/af2.obabel_hxr.autosite...,...,0.000000,1.000000,4.0,13.0,47.0,234.0,1.531915,0.505061,NaN,NaN


In [13]:
cols_ = ['struct_id', 'pocket_id', #'pocket_pLDDT_mean', 
    'pocket_score_combined_scaled', 'pocket_resid_swissmodel_coverage', #'pocket_resid', 'struct_resid', 'struct_resid_pathogenic', 
    'clinvar_path', 'ddd_path', 'pharmgkb',
    'n_pocket_resid_pathogenic', 'n_pocket_resid_not_pathogenic', 'n_others_resid_pathogenic', 'n_others_resid_not_pathogenic', 'pathogenic_odds', 'pathogenic_pvalue', 'pathogenic_padj',
    'n_pocket_resid_benign', 'n_pocket_resid_not_benign', 'n_others_resid_benign', 'n_others_resid_not_benign', 'benign_odds', 'benign_pvalue', 'benign_padj',
]

#df_stats_q_ = df_pval_.query('resid_pdb != resid_pdb & resid_swiss != resid_swiss').sort_values('pvals_corrected', ascending=True)
#df_stats_q_ = df_pval_.query('pocket_resid_swissmodel_coverage != "full"').sort_values('pvals_corrected', ascending=True)
#df_stats_q_ = df_pval_.query('resid_pdb != resid_pdb').sort_values('pvals_corrected', ascending=True)
#df_stats_q_ = df_pval_.query('ddd_path != ddd_path').sort_values('pvals_corrected', ascending=True)
#df_stats_q_ = df_pval_.query('pharmgkb == pharmgkb & n_others_resid_pathogenic == 0 & pocket_id < 5').sort_values('pvals_corrected', ascending=True)
#df_stats_q_ = df_pval_.query('pvals_corrected < .05').sort_values('pvals_corrected', ascending=True)
#df_stats_q_ = df_pval_.query('pvals_corrected < .05 & pocket_resid_swissmodel_coverage == "none"').sort_values('pvals_corrected', ascending=True)
df_stats_q_ = df_.query('struct_id == "P12235-F1"')
printlen(df_stats_q_)
df_stats_q_[cols_].head(20)

19


,struct_id,pocket_id,pocket_score_combined_scaled,pocket_resid_swissmodel_coverage,clinvar_path,ddd_path,pharmgkb,n_pocket_resid_pathogenic,n_pocket_resid_not_pathogenic,n_others_resid_pathogenic,...,pathogenic_odds,pathogenic_pvalue,pathogenic_padj,n_pocket_resid_benign,n_pocket_resid_not_benign,n_others_resid_benign,n_others_resid_not_benign,benign_odds,benign_pvalue,benign_padj
103968,P12235-F1,1,984.849729,partial,"{80, 33, 123, 278}",{},{},3.0,35.0,1.0,...,22.200000,0.007041,0.042244,4.0,34.0,47.0,213.0,0.533166,0.355965,NaN
103969,P12235-F1,2,892.829307,partial,"{80, 33, 123, 278}",{},{},0.0,18.0,4.0,...,0.000000,1.000000,NaN,2.0,16.0,49.0,231.0,0.589286,0.747582,NaN
103970,P12235-F1,3,866.287898,partial,"{80, 33, 123, 278}",{},{},0.0,23.0,4.0,...,0.000000,1.000000,NaN,4.0,19.0,47.0,228.0,1.021277,1.000000,NaN
103971,P12235-F1,4,888.740469,partial,"{80, 33, 123, 278}",{},{},0.0,17.0,4.0,...,0.000000,1.000000,NaN,3.0,14.0,48.0,233.0,1.040179,1.000000,NaN
103972,P12235-F1,5,837.816334,partial,"{80, 33, 123, 278}",{},{},0.0,24.0,4.0,...,0.000000,1.000000,NaN,1.0,23.0,50.0,224.0,0.194783,0.092594,NaN
103973,P12235-F1,6,870.961724,partial,"{80, 33, 123, 278}",{},{},0.0,16.0,4.0,...,0.000000,1.000000,NaN,2.0,14.0,49.0,233.0,0.679300,1.000000,NaN
103974,P12235-F1,7,750.970980,partial,"{80, 33, 123, 278}",{},{},0.0,16.0,4.0,...,0.000000,1.000000,NaN,2.0,14.0,49.0,233.0,0.679300,1.000000,NaN
103975,P12235-F1,8,423.679117,partial,"{80, 33, 123, 278}",{},{},0.0,16.0,4.0,...,0.000000,1.000000,NaN,3.0,13.0,48.0,234.0,1.125000,0.742359,NaN
103976,P12235-F1,9,499.289774,partial,"{80, 33, 123, 278}",{},{},1.0,14.0,3.0,...,6.666667,0.187519,NaN,3.0,12.0,48.0,235.0,1.223958,0.727357,NaN
103977,P12235-F1,10,823.668684,partial,"{80, 33, 123, 278}",{},{},0.0,17.0,4.0,...,0.000000,1.000000,NaN,4.0,13.0,47.0,234.0,1.531915,0.505061,NaN


In [14]:
r = df_stats_q_.query('struct_id == "P12235-F1" & pocket_id == 1').squeeze()
#r = df_stats_q_.query('struct_id == "P00325-F1" & pocket_id == 1').squeeze()
to_pymol_init()
to_pymol_struct(r.struct_id)
to_pymol_pocket(r.struct_id, r.pocket_id, color='tab_pink')

print(f'color tab_blue, {r.struct_id} & resid', pymol_resid(df_benign.loc['P12235'].squeeze()))
print(f'color tab_red, {r.struct_id} & resid', pymol_resid(r.pocket_resid & r.pathogenic_resid))
print(f'color tab_orange, {r.struct_id} & resid', pymol_resid(r.pathogenic_resid - r.pocket_resid))
print(f'show sticks, {r.struct_id} & resid', pymol_resid(r.pocket_resid & r.pathogenic_resid))
print(f'show sticks, {r.struct_id} & resid', pymol_resid(r.pathogenic_resid - r.pocket_resid))

delete all
bg_color white
set_color tab_blue, [31,119,180]
set_color tab_orange, [255,127,14]
set_color tab_green, [44,160,44]
set_color tab_red, [214,39,40]
set_color tab_purple, [148,103,189]
set_color tab_brown, [140,86,75]
set_color tab_pink, [227,119,194]
set_color tab_gray, [127,127,127]
set_color tab_olive, [188,189,34]
set_color tab_cyan, [23,190,207]
load ~/euler-home/project/22.12_pocketomes/results/23.04_bindfunc/af2/P1/22/35/P12235-F1.pdb
color grey50, P12235-F1
load ~/euler-home/project/22.12_pocketomes/results/23.04_bindfunc/af2.obabel_hxr.autosite/P1/22/35/P12235-F1/P12235-F1_cl_001.pdb
hide everything, P12235-F1_cl_001
show surface, P12235-F1_cl_001
set transparency, 0.5, P12235-F1_cl_001
color tab_pink, P12235-F1_cl_001
color tab_blue, P12235-F1 & resid 4+9+17+20+24+25+27+32+43+56+63+81+84+89+92+108+113+115+117+118+123+125+139+141+145+148+149+157+164+167+172+182+189+192+193+209+216+217+218+221+227+229+237+247+252+264+283+287+288+289+294
color tab_red, P12235-F1 & resid

In [15]:
print(len(r.pocket_resid & r.pathogenic_resid), 'of', len(r.pathogenic_resid), 'residues with a pathogenic variant in the pocket')
print(len(r.pocket_resid & df_benign.loc['P12235'].squeeze()), 'of', len(df_benign.loc['P12235'].squeeze()), 'residues with a benign variant in the pocket')

3 of 4 residues with a pathogenic variant in the pocket
4 of 51 residues with a benign variant in the pocket


In [16]:
#cols_ = ['Gene', 'Drug(s)', 'Phenotype(s)', 'Phenotype Category', 'Uniprot_canonical_isoform_(non_canonical)', 'Amino_acid_position', 'URL']
#q_ = f'`Uniprot_canonical_isoform_(non_canonical)` == "{strip_fragment_id(r.struct_id)}"'
#df_pharmgkb.query(q_)[cols_]

In [17]:
#df_ddd.query(f'uniprot_id == "{strip_fragment_id(r.struct_id)}"')